In [86]:
import pandas as pd
import os
import json
from tqdm import tqdm
pd.set_option('max_colwidth', 400)

In [90]:
import pandas as pd
from scipy import spatial
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader
from scipy import stats
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, LoggingHandler, util, models, evaluation, losses, InputExample

# Leitura da entrada

In [66]:
def transform_json(diario_json):
    dict_list =  []
    for key, value in json.loads(diario_json).items():
        for v in value:
            v["origin"] = key
            dict_list.append(v)
    return pd.DataFrame(dict_list)

In [98]:
dfs = []
directory = "../saida_segmentacao/AMM/"
for filename in tqdm(list(os.listdir(directory))):
    with open(directory + filename, "r") as jsonfile:
        diario_json = jsonfile.read()
        dfs.append(transform_json(diario_json))

100%|█████████████████████████████████████████████████████████████████████████████| 1640/1640 [00:20<00:00, 80.83it/s]


In [99]:
diario_df = pd.concat(dfs)

In [100]:
diario_df.sample(3)

,titulo,subtitulo,materia,publicador,id,origin
191,LICITAÇÃO E CONTRATOS,PREGÃO PRESENCIAL Nº 034/2019,"Processo Licitatório n°071/2019 Através de seu Pregoeiro Municipal - Haroldo Lourenço da Rocha e sua Equipe de Apoio, comunica que fará realizar às 09:00 horas do dia 02 de agosto de 2019, na sede da Prefeitura Municipal de Itamarati de Minas, licitação tipo menor preço por item, que tem por objetivo a contratação de Empresa Seguradora para cobertura de seguro da frota de veículos da prefeitur...",Haroldo Lourenço da Rocha,7B0CA45D,PREFEITURA DE ITAMARATI DE MINAS
128,COMISSÃO DE LICITAÇÃO,EXTRATO DE CONTRATO,"Extrato de Contrato nº 007/2016 - Processo Licitatório nº 384/2015 - Pregão Presencial Nº 053/2015 – Objeto: Aquisição de gêneros alimentícios para a merenda escolar. A Prefeitura Municipal de Japaraíba/MG torna público que celebrou Contrato com a empresa Açougue e Mercearia Andorinha Ltda - ME ao preço global de R$66.771,62 até 31/12/2016. Japaraíba/MG, 20/01/2016. ROBERTO EMILIO LOPES Prefei...",Roselene Maria Frazão,A0582115,PREFEITURA DE JAPARAIBA
281,"LICITAÇÕES, CONTRATOS, ATOS ADMINISTRATIVOS,",ETC ERRATA 001 - PREGÃO 064/2017,"MUNICÍPIO DE PIRANGUINHO – Retificação de Edital de Licitação - ERRATA 001 – PREGÃO PRESENCIAL Nº 064/2017 – PROCESSO: 119/2017- OBJETO:AQUISIÇÃO DE MATERIAIS DE CONSUMO E UTENSÍLIOS DOMÉSTICOS PARA SECRETARIA MUNICIPAL DE EDUCAÇÃO E SECRETARIA MUNICIPAL DE SAÚDE – EXCLUSIVO PARA ME, EPP e MEI. HOUVE ALTERAÇÃO NO OBJETO DA LICITAÇÃO – TERMO DE REFERÊNCIA, no que se refere ao quantitativo e esp...",Luciana Cristina Ribeiro,9904779B,PREFEITURA DE PIRANGUINHO


# Criação do dataset

In [133]:
def get_first_upper_tokens(text):
    result = " "
    for token in text.split():
        if token == token.upper():
            result += token + " "
        else:
            return result
    return result

def get_doc_text(text):
    first_tokens = get_first_upper_tokens(text)
    
    return text[len(first_tokens) - 1:]

def transform_df(diario_df):
    diario_df["query"] = diario_df.apply(lambda row:row["origin"] +" " +  row["titulo"] +" " + row["subtitulo"],axis = 1)
    diario_df["doc"] = diario_df["materia"]
    diario_df["doc"] = diario_df["materia"].apply(get_doc_text)
    diario_df["chamada"] = diario_df["materia"].apply(get_first_upper_tokens)
    diario_df["query"] = diario_df.apply(lambda row:row["query"] + row["chamada"],axis = 1)
    return diario_df[["query", "doc"]]

In [134]:
train_df = transform_df(diario_df)
train_df

,query,doc
0,ASSOCIAÇÃO DOS MUNICIPIOS DE BAIXA MOGIANA EDITAIS TERMO DE HOMOLOGAÇÃO E ADJUDICAÇÃO -PROCESSO LICITATÓRIO Nº: 03/2014 - PREGÃO Nº: 01/2014,"Pelo presente termo, levando em consideração o julgamento do Pregoeiro, que examinou as propostas referentes ao processo acima identificado, apontando como propostas mais vantajosas à Associação as apresentadas pelas empresas: CSC CONSTRUTORA SIQUEIRA CARDOSO LTDA, estabelecida na Rua dos Caetés, 285, CEP: 37.901-502, Passos/MG, CNPJ: 07.681.483/0001-86 e EXATI TECNOLOGIA E DESENVOLVIMENTO DE ..."
1,ASSOCIAÇÃO DOS MUNICIPIOS DE BAIXA MOGIANA Expediente:,Associação Mineira de Municípios – AMM - MG Diretoria 2011/2013 Presidente - Ângelo José Roncalli de Freitas 1º Vice-Presidente - Acácio Mendes de Andrade 2º Vice-Presidente - José Milton de Carvalho Rocha 3º Vice-Presidente - Marco Antônio de Andrade 1º Secretário - Aurelio Cezar Donadia Ferreira 2º Secretário - Eduardo Antônio Carvalho 1º Tesoureiro - José Sacido Bacia Neto 2º Tesoureiro - M...
2,ASSOCIAÇÃO DOS MUNICIPIOS DE BAIXA MOGIANA SETOR DE COMPRAS E LICITAÇÕES PREGÃO PRESENCIAL 03/2015 A,"Câmara Municipal de Pará de Minas, com sede à Avenida Presidente Vargas, 1935 nesta cidade de Pará de Minas/MG, por intermédio de sua Pregoeira, torna público, para conhecimento dos interessados, que fará realizar licitação com as seguintes características: MODALIDADE: Pregão Presencial Nº03/2015 TIPO: Menor preço por lote OBJETO: REGISTRO DE PREÇO para futuras e eventuais aquisições de gênero..."
3,CÂMARA MUNICIPAL DE RIO PARANAÍBA CÂMARA MUNICIPAL DE RIO PARANAÍBA PREGÃO PRESENCIAL Nº 01/2015. AVISO DE LICITAÇÃO PROCESSO LICITATÓRIO Nº 02/2015 PREGÃO PRESENCIAL Nº 01/2015,"OBJETO:Constitui objeto da presente licitação: Aquisição parcelada de combustível mediante requisição periódica, com especificações contidas no Anexo I - Termo de Referência do Edital. ·APRESENTAÇÃO PARA CREDENCIAMENTO DOS LICITANTES:Dia 09/02/2015, às 14h (quatorze horas). ·ABERTURA DA SESSÃO OFICIAL DO PREGÃO PRESENCIAL:Dia 09/02/2015, às 14h10m (quatorze horas e dez minutos). ·LOCAL PARA RE..."
4,PREFEITURA DE ALVORADA DE MINAS LICITAÇÃO EDITAL PREGAO PRESENCIAL 05/2015 A,"Prefeitura Municipal de Alvorada de Minas torna público que fará realizar Processo Licitatório Pregão Presencial n.º 0005/2014. Objeto: REFERENTE A LOCACAO DE TRES TRATORES AGRICOLAS COM IMPLEMENTO ( GRADE, ENSILADEIRA E CARROÇA) PARA ATENDIMENTO AO PROGRAMA DE INCENTIVO AO PEQUENO PRODUTOR RURAL CONFORME LEI MUNICIPAL 832/2013 PARA O ANO DE 2015.. Entrega dos envelopes: até às 09:00:00 de Ter..."
...,...,...
901,PREFEITURA MUNICIPAL DE VISCONDE DO RIO BRANCO DEPARTAMENTO DE CONVÊNIOS EXERCÍCIO DE 2020 EXTRATO DE PUBLICAÇÃO DISPENSA DE CHAMAMENTO PÚBLICO CELEBRAÇÃO DE TERMO DE FOMENTO N°. 002/2020 PARTES: MUNICÍPIO DE VISCONDE DO RIO BRANCO – MINAS GERAIS,e as OBRAS SOCIAIS DA PAROQUIA DE SÃO JOÃO BATISTA DE VISCONDE DO RIO BRANCO – CENTRO SOCIAL SÃO FRANCISCO DE ASSIS; CENTRO SOCIAL SANTA CLARA; LAR SÃO JOÃO BATISTA (LAR DOS IDOSOS). OBJETO: Firmar TERMO DE FOMENTO cujo objetivo é o repasse de recursos financeiros para as OBRAS SOCIAIS DA PAROQUIA DE SÃO JOÃO BATISTA DE VISCONDE DO RIO BRANCO – CENTRO SOCIAL SÃO FRANCISCO DE ASSIS; CENTRO SOCI...
902,PREFEITURA MUNICIPAL DE VISCONDE DO RIO BRANCO DEPARTAMENTO DE LICITAÇÃO E CONTRATOS EXTRATO DE AVISO DE PENALIDADE AVISO DE PENALIDADE – PREGÃO 058/2019 – PROCESSO LICITATÓRIO 108/2019 – ÓRGÃO GERENCIADOR: PREFEITURA MUNICIPAL DE VISCONDE DO RIO BRANCO – FORNECEDOR REGISTRADO: POLO COMERCIAL EIRELI– 1),"Aplicação de advertência à empresa de não mais participar de certames dos quais não possa cumprir com as obrigações assumidas e conforme as especificações do Edital, sob pena de aplicação de penalidade mais severa; 2) Anotação restritiva no cadastro de Fornecedores do Município; 3) Impedimento de licitar e contratar com a Prefeitura Municipal de Visconde do Rio Branco – MG, pelo prazo de 02 (d..."
903,SAAE - SERVIÇO AUTÔNOMO DE ÁGUA E ESGOTO 

# Treinamento

In [140]:
from sentence_transformers import SentenceTransformer, models
from torch import nn

word_embedding_model = models.Transformer('allenai/scibert_scivocab_uncased', max_seq_length=150)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [141]:
examples = []
for i in tqdm(range(1, len(train_df)), position=0, leave=True, total=len(train_df)):
  row, neg = train_df.iloc[i], train_df.iloc[i - 1]
  examples.append(InputExample(texts=[row["query"], row["doc"], neg["doc"]]))

100%|██████████████████████████████████████████████████████████████████████▉| 699758/699759 [01:48<00:00, 6463.68it/s]


In [142]:
false_pairs, true_pairs = 0, 0
SAMPLE_SIZE = 1000
df_sample = train_df.sample(SAMPLE_SIZE, random_state=42)
for i in tqdm(range(1, SAMPLE_SIZE)):
  row, neg = df_sample.iloc[i], df_sample.iloc[i - 1]

  output = model.encode([row["query"], row["doc"], neg["doc"]]), 

  true_pairs += 1 - spatial.distance.cosine(output[0][0], output[0][1])
  false_pairs += 1 - spatial.distance.cosine(output[0][0], output[0][2])
    
print("Média de similaridade dos positivos: ", true_pairs / SAMPLE_SIZE)
print("Média de similaridade dos negativos: ", false_pairs / SAMPLE_SIZE)

100%|███████████████████████████████████████████████████████████████████████████████| 999/999 [00:19<00:00, 50.42it/s]

Média de similaridade dos positivos:  0.8480542653501034
Média de similaridade dos negativos:  0.8405530993044377


In [143]:
dataloader = DataLoader(examples[:20000], shuffle=True, batch_size=8)
train_loss = losses.TripletLoss(model)

#Tune the model
model.fit(train_objectives=[(dataloader, train_loss)], epochs=1, warmup_steps=50)

In [144]:
false_pairs, true_pairs = 0, 0
SAMPLE_SIZE = 1000
df_sample = train_df.sample(SAMPLE_SIZE, random_state=42)
for i in tqdm(range(1, SAMPLE_SIZE)):
  row, neg = df_sample.iloc[i], df_sample.iloc[i - 1]

  output = model.encode([row["query"], row["doc"], neg["doc"]]), 

  true_pairs += 1 - spatial.distance.cosine(output[0][0], output[0][1])
  false_pairs += 1 - spatial.distance.cosine(output[0][0], output[0][2])

print("Média de similaridade dos positivos: ", true_pairs / SAMPLE_SIZE)
print("Média de similaridade dos negativos: ", false_pairs / SAMPLE_SIZE)

100%|███████████████████████████████████████████████████████████████████████████████| 999/999 [00:19<00:00, 50.19it/s]

Média de similaridade dos positivos:  0.706841331361793
Média de similaridade dos negativos:  0.2014226446911416
